### Comenzemos el ETL, en este archivo usaremos el dataset de credits.csv para extraer el nombre de los directores de cada pelicula, lo que nos ayudara a completar el trabajo junto a los endpoints.

In [175]:
#Importamos librerias necesarias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [176]:
#Leemos el dataset
df = pd.read_csv('data/credits.csv')
df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Podemos observar que en la columna de "crew" se encuentra el personal de la pelicula, y si miramos mas alla encontraemos que cada fila es una lista con diccionarios dentro, tendremos que iterar en cada uno para extraer los directores.

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


Nos dejemos solo la columna crew.

In [178]:
df_crew = df['crew']
df_crew.head()

0    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1    [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2    [{'credit_id': '52fe466a9251416c75077a89', 'de...
3    [{'credit_id': '52fe44779251416c91011acb', 'de...
4    [{'credit_id': '52fe44959251416c75039ed7', 'de...
Name: crew, dtype: object

y creemos una funcion para extraer los directores.

In [1]:
#lista vacia donde iran los directores
directores = []
#iteramos sobre el total del dataframe
for i in range(0, df_crew.shape[0]):
    #hacemos un eval para tomar como lista y no string
    lista = eval(df_crew.iloc[i])
    #iniciamos una lista para ingresar los directores de la misma pelicula
    lista_directores = []
    #iteramos en los diccionarios
    for dicc in lista:
        #si el trabajo es director
        if dicc['job'] == 'Director':
            #ingresamos el nombre a la lista de la pelicula
            lista_directores.append(dicc['name'])
    #y por ultimo ingresamos las listas a los directores.
    directores.append(lista_directores)

#creamos un diccionario de los directores
dicc_directores = {'director': directores}
#para convertirlo en un dataframe
df_directores = pd.DataFrame(dicc_directores)
df_directores

NameError: name 'df_crew' is not defined

In [180]:
#agregamos directores a nuestro set
df['directores'] = df_directores
#y dropeamos el resto
df.drop(['cast', 'crew'], axis=1, inplace=True)
df.head()

,id,directores
0,862,[John Lasseter]
1,8844,[Joe Johnston]
2,15602,[Howard Deutch]
3,31357,[Forest Whitaker]
4,11862,[Charles Shyer]


Ahora que tenemos una dataframe con los directores y su id de pelicula podemos hacer un merge, traigamos el otro dataset para unirlos.

In [181]:
df_movies = pd.read_csv('data/movies_dataset.csv', low_memory=False)
df_movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [182]:
#cambiamos el tipo de 'id' ya que lo tomo como entero.
df['id'] = df['id'].astype('str')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          45476 non-null  object
 1   directores  45476 non-null  object
dtypes: object(2)
memory usage: 710.7+ KB


In [2]:
#y por ultimo hacemos el merge.
df_final = pd.merge(df_movies, df, on='id', how='outer', indicator=True)

NameError: name 'pd' is not defined

In [184]:
df_final.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,directores,_merge
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,[John Lasseter],both
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,[Joe Johnston],both


In [185]:
#eliminamos columna creada en el merge
df_final.drop('_merge', axis=1, inplace=True)


Y por ultimo extraemos el dataset nuevo para su posterior ETL junto al dataframe.

In [186]:
df_final.to_csv('data/movies_directores.csv')